#### Getting setup

In [60]:
import xarray as xr
import numpy as np  
from scipy import stats    
from xmhw.xmhw import threshold, detect
import copernicusmarine as cmems

In [67]:
from bokeh.plotting import figure, show
from bokeh.models import (
    CheckboxButtonGroup,
    Div,
    CustomJS,
    Switch,
    HoverTool,
    ColumnDataSource,
    PanTool,
    BoxZoomTool,
    WheelZoomTool,
    ResetTool,
    Slider,
    RadioButtonGroup,
    Legend,
    LinearAxis,
    Range1d
)
from bokeh.io import output_notebook, output_file
from bokeh.layouts import row, column
from bokeh.transform import dodge

output_notebook()

# Define the main plotting colours we will use
c_blue = '#00c2ce'
c_red = '#fe5900'

Loading BokehJS ...

**Downloading data**

The data for this notebook will be downloaded from the Copernicus Marine Environment Monitoring Service (CMEMS) datastore. A free account is required to access the datastore, and can be created [here](https://data.marine.copernicus.eu/register). Make a note of your username and password.

The next cell authenticates the connection to the CMEMS datastore. It only needs to be run once to create the credentials file locally. Change the username and password fields to your CMEMS credentials, stored as strings. For example if your username is `Copernicus1` and your password is `1234`, then write `username = "Copernicus1"` and `password = "1234"`. Once you've run the cell once, change the `login` parameter to `False` to prevent issues rerunning the notebook.

In [ ]:
# NOTE: Running this cell once the credentials file has already been created can cause issues. Please change
# the `login` parameter to `False` once the file has been created to avoid issues rerunning the notebook.

login = False  # Change to False once credentials file has been created

username = "username"  # Change to your CMEMS username
password = "password"  # Change to your CMEMS password

if login:
    cmems.login(username=username, password=password)

The following cell specifies the dataset we will access: [Global Ocean OSTIA Sea Surface Temperature and Sea Ice Reprocessed](https://data.marine.copernicus.eu/product/SST_GLO_SST_L4_REP_OBSERVATIONS_010_011/description), as well as the region of interest (ROI), between 36 and 42 degrees North and 10 and 15 degrees East, and the dates of interest, between 1982 and 2021 inclusive.

In [15]:
# Set tags to identify datasets
tags = ["MY"]

# CMEMS multi-year reprocessed (MY) SST identifier
dataset_my = "METOFFICE-GLO-SST-L4-REP-OBS-SST"
savefile_my = f"SST_{dataset_my}"

# L4 time series ROI
ROI_latitudes = [36, 42]
ROI_longitudes = [10, 15]

# climatology
start_year = 1982
end_year = 2021

To download the full dataset associated to the above parameters would require over 2600MB of storage. We are only interested in the regional average of SST across the ROI, so storing all that data locally is not needed. Instead, we can access and query the dataset remotely on the CMEMS datastore using the function `open_dataset`. The following cell opens the specified dataset and stores it as `ds`.

In [6]:
ds = cmems.open_dataset(
    dataset_id=dataset_my,
    minimum_longitude=ROI_longitudes[0],
    maximum_longitude=ROI_longitudes[1],
    minimum_latitude=ROI_latitudes[0],
    maximum_latitude=ROI_latitudes[1],
    start_datetime= f"{start_year}-01-01",
    end_datetime= f"{end_year}-12-31",
)

INFO - 2024-02-15T10:17:45Z - Dataset version was not specified, the latest one was selected: "202003"
INFO - 2024-02-15T10:17:45Z - Dataset part was not specified, the first one was selected: "default"
INFO - 2024-02-15T10:17:46Z - Service was not specified, the default one was selected: "arco-time-series"


You can inspect the dataset `ds` using `display(ds)`. You will see it contains 3 coordinates: `latitude`, `longitude` and `time`, and 4 data variables, including `analysed_sst`. We are interested in the spatial average over time of the SST data. The following cell creates a time series `ts` by taking the mean over `latitude` and `longitude` of the `analysed_sst` variable.

In [18]:
download_data = False  # Change to False once the data is downloaded to prevent the cell running again

if download_data:
    # Compute the regional mean of the analysed_sst variable
    ts = ds.analysed_sst.mean(dim=["latitude", "longitude"])

    # Store the time series ts locally. By default, it is stored in the same directory as your
    # notebook is running from.
    ts.to_netcdf(savefile_my)

**Storing data in a dictionary**

Once all the data has been downloaded, the final step to getting setup is to store the data. Throughout this task, we'll build up a dictionary `heatwaves` which will keep track of all the data we generate. We'll start by creating a sub-dictionary `heatwaves["MY"]` and storing the MY data in the `"SST"` attribute.

In [20]:
heatwaves = {}
for tag in tags:
    heatwaves[tag] = {}
    ts = xr.open_dataarray(savefile_my)
    heatwaves[tag]["SST"] = ts
    ts.close()


If you ever get lost, use the `keys()` function to access what each part of the dictionary stores. For example, `heatwaves["MY"].keys()` shows that `heatwaves["MY"]` contains the attribute `SST`.

#### Lesson 1: Marine heatwaves

The goal of this task is to analyse marine heatwaves (MHWs) for a given set of sea surface temperature (SST data). We will follow the definition given by [Hobday et al. (2016)](https://www.sciencedirect.com/science/article/abs/pii/S0079661116000057). The linked paper gives justifications for parameter choices in the definition, as well as defining characteristics of MHWs such as intensity and onset rate. These characteristics are not covered in this course, but you will be able to analyse them using the tools covered.


**Definition**

Marine heatwaves are defined by having all three of the following three features.

**Anomalously warm period**

An anomalously warm period is defined as when recorded temperature is above a percentile threshold of given climate data.
* The suggested percentile is 90%.
* This avoids assumptions about the distribution of anomalies, unlike standard deviation.
* The definition depends on the choice of climatology.

**Prolonged period**

The temperature must remain above the threshold for a certain number of days to be considered as a heatwave.
* The suggested duration is 5 days. This is set to balance the detection of heatwaves in tropical locations (elevated with shorter duration) and in cooler locations (disproportionately diminished with longer duration).

**Discrete period**

A period of cooler days must exist between two periods of elevated temperatures to distinguish them as different heatwaves.
*The suggested period is 2 days.

#### Lesson 2: Defining and analysing climatology

**MY data climatology**

The `xmhw` python package contains all the functionality we need to detect and analyse MHWs. We will first use it to generate climatologies for our datasets. The `xmhw` function `threshold` computes the climatology for a given time series `TS` and stores it as the attribute `threshold(TS).seas`. For further discussion of how the climatology is computed in this package, along with some of the options, please see "Detail on climatology" section below.

The following cell generates climatology for the MY dataet.

In [23]:
# Define the start and end years for the full climatology
clim_start_year = 1982
clim_end_year = 2021

# Access the multi-year dataset
tag = "MY"
dataset = heatwaves[tag]["SST"]

# Setup the dictionary for storing this climatology
heatwaves[tag]["full"] = {}
clim_dict = heatwaves[tag]["full"]

# Slice the dataset to the desired time range
clim_dict["time_series"] = (
    dataset.sel(time=slice(f"{clim_start_year}-01-01", f"{clim_end_year}-12-31"))
    - 273.15
)
clim_dict["start_year"] = clim_dict["time_series"][0].time.dt.year.values
clim_dict["end_year"] = clim_dict["time_series"][-1].time.dt.year.values

# Compute climatology
clim_dict["xmhw_thresh"] = threshold(clim_dict["time_series"])

The following cell then plots the generated climatology.

In [37]:
# Access the desired dataset
tag = "MY"
series = "full"

working_dict = heatwaves[tag][series]

output_file(f"Climatology_{tag}_{series}.html")

# Define data to plot
data = ColumnDataSource(
    data=dict(t=working_dict["xmhw_thresh"].doy, seas=working_dict["xmhw_thresh"].seas)
)

# Setup the plot and toolbar
tools = [BoxZoomTool(), WheelZoomTool(), PanTool(), ResetTool()]

p = figure(
    title=f"SST Climatology [climatology: {tag}, {working_dict['start_year']} - {working_dict['end_year']}]",
    height=400,
    width=1000,
    tools=tools,
    x_axis_label="Day of year",
    y_axis_label=r"\[ SST \, [^\circ C] \]",
)

p.toolbar.autohide = True

# Plot the climatology
p_seas = p.line("t", "seas", source=data, line_width=1.5, color="black")

# Add a hovertool and legend to the plot
p.add_tools(
    HoverTool(
        renderers=[p_seas],
        tooltips=[
            ("Day of year", "@t"),
            ("Climatology", "@seas"),
        ],
        mode="vline",
        toggleable=False,
    )
)

legend = Legend(
    items=[(f"Seasonal climatology [{tag}]", [p_seas])], location="top_right"
)
p.add_layout(legend, "center")

# Setup the plot space and display!

container = row(p, height=400, sizing_mode="stretch_both")
container.stylesheets.append(":host { width: 100%; }")

show(container)

Loading BokehJS ...

**Detail on climatology**

The `xmhw` package computes mean climatology through using a rolling average as a smoothing method. As the day of year (DoY) varies, a rolling average is taken using a window of data centred on the DoY of width `31`. This parameter can be changed to any _odd number_ n when calling the `xmhw.threshold` function by passing the keyword argument `smoothPercentileWidth = n`.

The following plot demonstrates the calculation of climatologies using different smoothing window widths from 1 day up to a whole year. Remember that you can zoom in on parts of the plot to see changes more clearly.


In [38]:
output_file("detail_on_climatology.html")

# Select data for the plot
tag = "MY"
series = "full"

working_dict = heatwaves[tag][series]

# Setup the smoothing windows to visualise and interaction widget

smoothing_options = ["1", "3", "7", "15", "31", "91", "183", "365"]

cbbg = RadioButtonGroup(labels=smoothing_options, active=4)

# Setup the basic plot, and add the SST and climatology graphs

tools = [BoxZoomTool(), WheelZoomTool(), PanTool(), ResetTool()]

p = figure(
    title=f"Climatology with varything smoothing window widths "
    f"[Climatology data: {tag}, {working_dict['start_year']} - {working_dict['end_year']}]",
    height=400,
    width=1000,
    tools=tools,
    x_axis_label="Day of year",
    y_axis_label=r"\[ SST \, [^\circ C] \]",
)

# Run through the percentiles in the defined range. For each, plot the threshold
# graph, as well as the time periods of detected MHWs for that threshold

p_clims = {}
for i, width in enumerate(smoothing_options):
    # Initially, only the plots corresponding to the lowest percentile are visible

    visible = True if cbbg.active == i else False

    # Compute the threshold for the given percentile

    clim = threshold(working_dict["time_series"], smoothPercentileWidth=int(width))

    # Collect the threshold data and plot the threshold graph

    data = ColumnDataSource(data=dict(t=clim.doy, seas=clim.seas))

    p_clims[width] = [
        i,
        p.line(
            "t",
            "seas",
            source=data,
            line_width=1.5,
            color="black",
            visible=visible,
        ),
    ]

    # Add a hovertool to the threshold plot capturing climatology data

    p.add_tools(
        HoverTool(
            renderers=[p_clims[width][1]],
            tooltips=[
                ("Day of year", "@t"),
                ("Climatology", "@seas"),
            ],
            mode="vline",
            toggleable=False,
        )
    )


# Link the radio button selection to the visibility of the correct plot

cbbg.js_on_change(
    "active",
    CustomJS(
        args=dict(cbbg=cbbg, plots=p_clims),
        code="""
            for (let plot in plots) {plots[plot][1].visible = cbbg.active == plots[plot][0]};
            """,
    ),
)

# Define and format the plot legend

legend = Legend(
    items=[(f"Seasonal climatology [{tag}]", [p_clims[p][1] for p in p_clims])],
    location="top_right",
)
p.add_layout(legend, "center")

# Define and format the full plot container and layout

cbbg_div = Div(text="Smoothing window width (days): ")

container = row(p, height=400, sizing_mode="stretch_width")
container.stylesheets.append(":host { width: 100%; }")

layout = column(row(cbbg_div, cbbg), container, sizing_mode="stretch_both")

# Display the plot!

show(layout)

As the smoothing window width increases, you can see the corresponding smoothing on the plot, especially around the maximum and minimum values. When the width is very large, however, the seasonal variation in temperatures starts to get smoothed out.

Since marine heatwaves should be sensitive to seasonal climatology (Hobday et al.), that is they should be detected in both cooler and warmer periods of the year, is the default value of `smoothPercentileWidth = 31` a good choice to achieve this?

#### Lesson 3: Comparing climatologies

**Comparing reference periods**

The climatology for MY data produced in the previous lesson was based on the 40-year reference period from 1982 - 2021. Often, climatology is based on shorter reference periods: for example, a typical climate data record (CDR) normally has a 30-year reference period. As part of this course, we will examine the impact that different climatologies has on marine heatwave detection. Therefore, we will begin by examining different climatologies produced from the same dataset over different reference periods. 

It will help to be able to more easily create slices of time series data, which the following function does. If `end_year` is left blank, it will produce a single year slice.

In [34]:
def create_TS_slice(tag, series_name, start_year, end_year=None):
    heatwaves[tag][series_name] = {}
    working_dict = heatwaves[tag][series_name]
    working_dict["time_series"] = (
        heatwaves[tag]["SST"].sel(
            time=slice(f"{start_year}-01-01", f"{end_year or start_year}-12-31")
        )
        - 273.15
    )
    working_dict["start_year"] = start_year
    working_dict["end_year"] = end_year or start_year

We will compare MY climatology from two 30-year reference periods: 1982 - 2011 and 1992 - 2021. The following cell slices the full MY time series to these dates, and computes climatology.

In [63]:
tag = "MY"

for start_year, end_year in zip([1982,1992], [2011,2021]):
    series_name = f"{start_year} - {end_year}"  
    create_TS_slice(tag, series_name, start_year, end_year)

    clim_dict = heatwaves[tag][series_name]
    clim_dict["xmhw_thresh"] = threshold(clim_dict["time_series"])


Next, we will plot the two climatologies on the same axes for comparison.

In [72]:
output_file("climatology_compare_MY_30_years.html")

# Access the desired data and prepare it for plotting
tag = "MY"
my_1 = "1982 - 2011"
my_2 = "1992 - 2021"

dict_my1 = heatwaves[tag][my_1]
dict_my2 = heatwaves[tag][my_2]

data = ColumnDataSource(
    data=dict(
        t=dict_my1["xmhw_thresh"].doy,
        MY_1=dict_my1["xmhw_thresh"].seas,
        MY_2=dict_my2["xmhw_thresh"].seas,
    )
)

# Setup the plot area, tools, and interaction buttons
tools = [BoxZoomTool(), WheelZoomTool(), PanTool(), ResetTool()]

labels = [my_1, my_2]
cbbg = CheckboxButtonGroup(labels=labels, active=[0, 1])

p = figure(
    title=(f"SST Climatology [datasets: MY, "
        f"{dict_my1['start_year']} - {dict_my1['end_year']}; "
        f"MY, {dict_my2['start_year']} - {dict_my2['end_year']}]"),
    height=400,
    width=1000,
    tools=tools,
    x_axis_label="Day of year",
    y_axis_label=r"\[ SST \, [^\circ C] \]",
)

p.toolbar.autohide = True

# Plot both MY climatologies
p_my1 = p.line(
    x="t",
    y="MY_1",
    source=data,
    line_width=1.5,
    color=c_blue,
    visible=0 in cbbg.active,
)
p_my2 = p.line(
    x="t",
    y="MY_2",
    source=data,
    line_width=1.5,
    color=c_red,
    visible=1 in cbbg.active,
)

# Add hovertools to the plot area
p.add_tools(
    HoverTool(
        renderers=[p_my1],
        tooltips=[
            ("Day of year", "@t"),
            (f"Climatology [{my_1}]", "@MY_1"),
        ],
        attachment="below",
        mode="vline",
        toggleable=False,
    ),
    HoverTool(
        renderers=[p_my2],
        tooltips=[("Day of year", "@t"), (f"Climatology [{my_2}]", "@MY_2")],
        attachment="above",
        mode="vline",
        toggleable=False,
    ),
)

# Add a legend to the plot
legend = Legend(
    items=[
        (f"Seasonal climatology [{my_1}]", [p_my1]),
        (f"Seasonal climatology [{my_2}]", [p_my2]),
    ],
    location="top_left",
)
p.add_layout(legend, "center")

# Add the JavaScript element controlling the plot visibility controlled by the button group
cbbg.js_on_change(
    "active",
    CustomJS(
        args=dict(p_my1=p_my1, p_my2=p_my2, cbbg=cbbg),
        code="""p_my1.visible = cbbg.active.includes(0), p_my2.visible = cbbg.active.includes(1)""",
    ),
)

# Compile the plot layout and display!
container = row(column(cbbg, p), height=450, sizing_mode="stretch_both")
container.stylesheets.append(":host { width: 100%; }")

show(container)

The plot clearly shows that the climatology generated from the 1992-2021 reference period is always greater than the climatology generated from 1982-2011.

What these plots don't show, however, is what has caused the difference between the two climatologies: all they tell us is that average temperatures in the period 1982-2011 were lower than in the period 1992-2011. Is this due to an upward trend in temperatures, or, for example, were there some very cold years in the period 1982-2011 which dragged down the full climatology?

Our intuition and experience tells us that it should be the former. Let's test that by performing a regression analysis on the full MY time series data. The following cell uses the inbuilt Python `stats` package to perform a regression of annual mean temperatures.

In [69]:
# Access the desired data
tag = "MY"
series = "full"

working_dict = heatwaves[tag][series]

# Compute the mean temperature per year by first grouping the time series per year,
# then taking the mean along the time dimension.
annual_mean = working_dict["time_series"].groupby('time.year').mean('time')

# Use the stats Python package to compute a regression model
regress = stats.linregress(annual_mean.year, annual_mean)

The generated object `regress` contains the gradient and intercept of the regression line as `regress.slope` and `regress.intercept`, respectively. You can also print the whole regress object to see what other data it contains. For example, the analysis also gives a `pvalue` parameter, the result of a significance test on the regression calculation.

We now plot the regression line on top of the full time series.

In [70]:
output_file("linear_regression_MY_full.html")

# Access the desired data and prepare it for plotting
tag = "MY"
series = "full"

working_dict = heatwaves[tag][series]

data = ColumnDataSource(
    data=dict(t=working_dict["time_series"].time, SST=working_dict["time_series"])
)

# Setup the plot area and tools
tools = [BoxZoomTool(), WheelZoomTool(), PanTool(), ResetTool()]

p = figure(
    title=f"Full time series with annual regression trend [dataset: {tag}]",
    height=400,
    width=1000,
    tools=tools,
    x_axis_type="datetime",
    x_axis_label="Date",
    y_axis_label=r"\[ SST \, [^\circ C] \]",
)

# Plot the full SST time series data
p_sst = p.line("t", "SST", source=data, line_width=1, color="black")

# Prepare the regression data for plotting
regress_data = ColumnDataSource(
    data=dict(
        year=np.array([np.datetime64(f"{y}-07-01") for y in annual_mean.year.values]),
        mean=annual_mean,
        reg=regress.intercept + regress.slope * annual_mean.year,
    )
)

# Plot the annual mean data and regression line
p_scatter = p.circle(
    "year",
    "mean",
    source=regress_data,
    fill_color=None,
    line_color=c_red,
    size=7,
    line_width=1.5,
)

p_reg = p.line(
    "year", "reg", source=regress_data, color=c_red, line_dash="dashed", line_width=1.5
)

# Add hovertools to the plot
p.add_tools(
    HoverTool(
        renderers=[p_sst],
        tooltips=[
            ("Date", "@t{%F}"),
            ("SST", "@SST"),
        ],
        mode="vline",
        formatters={"@t": "datetime"},
        toggleable=False,
    ),
    HoverTool(
        renderers=[p_scatter],
        tooltips=[("Year", "@year{%Y}"), ("Mean", "@mean")],
        formatters={"@year": "datetime"},
        toggleable=False,
    ),
    HoverTool(
        renderers=[p_reg],
        tooltips=[("Regression", f"{regress.intercept:.3f} + {regress.slope:.3f}*t")],
        toggleable=False,
    ),
)

# Define a switch and link it to toggle the visibility of the annual mean and regression line
reg_switch = Switch(active=True)

reg_switch.js_link("active", p_reg, "visible")
reg_switch.js_link("active", p_scatter, "visible")

reg_switch_div = Div(text="Toggle regression line:")

# Add the plot legend
legend = Legend(
    items=[
        ("SST", [p_sst]),
        ("Annual mean", [p_scatter]),
        ("Regression", [p_reg]),
    ],
    location="top_right",
    ncols=3,
)
p.add_layout(legend, "center")

# Compile the plot layout and display!
container = row(p, height=400, sizing_mode="stretch_width")
container.stylesheets.append(":host { width: 100%; }")

layout = column(row(reg_switch_div, reg_switch), container, sizing_mode="stretch_both")

show(layout)

#### Lesson 4: Detecting marine heatwaves

In this lesson, we will use our generated climatologies to detect marine heatwaves from time series data. Specifically, we will:

* Learn the process of plotting the MHW detection threshold for a given climatology;
* Learn to produce plots demonstrating the detection of MHWs.

Optional additional tasks will help you appreciate how the detection of MHWs change by changing parameters associated to the definition, such as the detection threshold.

**Plotting detection thresholds**

Recall that MHWs are defined when recorded temperatures passes above a threshold defined from a certain percentile of climatology data.

We used the `xmhw` function `threshold` to produce mean climatologies from time series data. This function also produces the detection threshold. For further discussion of how this function works and how to adjust the detection threshold, see the "Detail on thresholds" section below.

The following cell plots the climatology and detection threshold for the full series of MY data.

In [39]:
output_file("threshold_MY_full.html")

# Access the desired data and prepare it for plotting
tag = "MY"
series = "full"

working_dict = heatwaves[tag][series]

data = ColumnDataSource(
    data=dict(
        t=working_dict["xmhw_thresh"].doy,
        seas=working_dict["xmhw_thresh"].seas,
        thresh=working_dict["xmhw_thresh"].thresh,
    )
)

# Prepare the plot area and tools
tools = [BoxZoomTool(), WheelZoomTool(), PanTool(), ResetTool()]

p = figure(
    title=f"SST Climatology and MHW detection threshold "
    f"[climatology: {tag}, {working_dict['start_year']} - {working_dict['end_year']}]",
    height=400,
    width=1000,
    tools=tools,
    x_axis_label="Day of year",
    y_axis_label=r"\[ SST \, [^\circ C] \]",
)

p.toolbar.autohide = True

# Plot the climatology and detection threshold
p_seas = p.line(
    x="t", y="seas", source=data, line_width=1.5, color="black", line_dash="dashed"
)
p_thresh = p.line(x="t", y="thresh", source=data, line_width=1.5, color=c_blue)

# Add hovertools to the plots
p.add_tools(
    HoverTool(
        renderers=[p_thresh],
        tooltips=[
            ("Day of year", "@t"),
            ("MHW Threshold (90%)", "@thresh"),
        ],
        mode="vline",
        attachment="above",
        toggleable=False,
    ),
    HoverTool(
        renderers=[p_seas],
        tooltips=[
            ("Day of year", "@t"),
            ("Climatology [MY]", "@seas"),
        ],
        mode="vline",
        attachment="below",
        toggleable=False,
    ),
)

# Add the plot legend
legend = Legend(
    items=[
        (f"Seasonal climatology [MY]", [p_seas]),
        (f"90% MHW detection threshold", [p_thresh]),
    ],
    location="top_left",
)

p.add_layout(legend, "center")

# Compile the plot layout and display!
container = row(p, height=400, sizing_mode="stretch_both")
container.stylesheets.append(":host { width: 100%; }")

show(container)

**Superimposing time series data**

Next, we will plot our climatology and threshold against the actual time series data. Let's create a slice of the MY dataset over its last 5 years of recording, 2017-2021, and save it under the series name `"2017-2021"`. We'll then plot this slice against the full MY climatology and threshold.

In [35]:
tag = "MY"
series_name = "2017-2021"

create_TS_slice(tag, series_name, 2017, 2021)

In [41]:
output_file("time_series_threshold_MY_2017_2021.html")

# Select the desired data and prepare it for plotting
tag = "MY"
series = "2017-2021"

working_dict = heatwaves[tag][series]
clim_dict = heatwaves[tag]["full"]

data = ColumnDataSource(
    data=dict(
        t=working_dict["time_series"].time,
        SST=working_dict["time_series"],
        thresh=clim_dict["xmhw_thresh"].thresh[
            working_dict["time_series"].time.dt.dayofyear - 1
        ],
        seas=clim_dict["xmhw_thresh"].seas[
            working_dict["time_series"].time.dt.dayofyear - 1
        ],
    )
)

# Prepare the plotting space and tools
tools = [BoxZoomTool(), WheelZoomTool(), PanTool(), ResetTool()]

p = figure(
    title=f"SST, Climatology and MHW detection threshold "
    f"[climatology: {tag}, {clim_dict['start_year']} - {clim_dict['end_year']}]",
    height = 400,
    width = 1000,
    tools=tools,
    x_axis_type="datetime",
    x_axis_label="Date",
    y_axis_label=r"\[ SST \, [^\circ C] \]",
    y_range = (13.5,32),
)

p.toolbar.autohide = True

# Plot SST, threshold and climatology data
p_sst = p.line("t", "SST", source=data, line_width=1.5, color="black")

p_thresh = p.line("t", "thresh", source=data, line_width=1.5, color=c_blue)
p_seas = p.line(
    "t", "seas", source=data, line_width=1.5, color="black", line_dash="dashed"
)

# Add a hovertool to the plot
p.add_tools(
    HoverTool(
        renderers=[p_thresh],
        tooltips=[
            ("Date", "@t{%F}"),
            ("Threshold", "@thresh"),
            ("SST", "@SST"),
            ("Climatology", "@seas"),
        ],
        mode="vline",
        formatters={"@t": "datetime"},
    )
)

# Add the plot legend
legend = Legend(
    items=[
        (f"SST [MY]", [p_sst]),
        (f"Seasonal climatology [MY]", [p_seas]),
        (f"90% MHW detection threshold", [p_thresh]),
    ],
    location="top_right",
    ncols = 3,
)
p.add_layout(legend, "center")

# Compile the plot layout and display!
container = row(p, height=400, sizing_mode="stretch_both")
container.stylesheets.append(":host { width: 100%; }")

show(container)

Looking at the plot, we see some clear candidates for heatwaves, the first being in early 2017. Additionally, there are spikes in each summer. It is not clear, however, whether these satisfy the *prolonged period* part of the definition of a MHW. In the next section, we will use the `xmhw` package to help us detect which of these candidates are heatwaves.

**Detecting marine heatwaves**

The `xmhw` package contains functionality to detect heatwaves. For a given time series `TS`, climatology `clim` and detection threshold `thresh`, the function `detect(TS, thresh, clim)` produces an object storing all detected heatwaves according to the definition. For more information on this object, as well as how to change some of the parameters used for detection, such as the required duration, see the "Detail on MHW detection" section below.

Let's see the `detect` function in action for the above example.

In [43]:
tag = "MY"
series_name = "2017-2021"

working_dict = heatwaves[tag][series_name]
clim_dict = heatwaves[tag]["full"]

mhws = detect(
    working_dict["time_series"],
    clim_dict["xmhw_thresh"].thresh,
    clim_dict["xmhw_thresh"].seas,
)

mhw_start_times = mhws.time_start.values.astype("datetime64[D]").tolist()
mhw_end_times = mhws.time_end.values.astype("datetime64[D]").tolist()
mhw_number = min(len(mhw_start_times), len(mhw_end_times))

for i in range(mhw_number):
    start = mhw_start_times[i]
    end = mhw_end_times[i]
    print(
        f"Heatwave {i+1} of {mhw_number}: "
        f"\n start: {start.year}-{start.month}-{start.day}, end: {end.year}-{end.month}-{end.day}"
    )

Heatwave 1 of 26: 
 start: 2017-3-19, end: 2017-4-19
Heatwave 2 of 26: 
 start: 2017-5-31, end: 2017-6-6
Heatwave 3 of 26: 
 start: 2017-6-11, end: 2017-6-30
Heatwave 4 of 26: 
 start: 2017-8-4, end: 2017-8-10
Heatwave 5 of 26: 
 start: 2018-4-20, end: 2018-5-1
Heatwave 6 of 26: 
 start: 2018-5-30, end: 2018-6-6
Heatwave 7 of 26: 
 start: 2018-8-2, end: 2018-8-22
Heatwave 8 of 26: 
 start: 2018-9-10, end: 2018-9-25
Heatwave 9 of 26: 
 start: 2019-6-25, end: 2019-7-13
Heatwave 10 of 26: 
 start: 2019-8-23, end: 2019-9-5
Heatwave 11 of 26: 
 start: 2019-9-16, end: 2019-9-22
Heatwave 12 of 26: 
 start: 2019-9-26, end: 2019-10-3
Heatwave 13 of 26: 
 start: 2019-10-25, end: 2019-11-6
Heatwave 14 of 26: 
 start: 2020-1-31, end: 2020-2-5
Heatwave 15 of 26: 
 start: 2020-2-10, end: 2020-3-6
Heatwave 16 of 26: 
 start: 2020-3-11, end: 2020-3-21
Heatwave 17 of 26: 
 start: 2020-4-9, end: 2020-4-13
Heatwave 18 of 26: 
 start: 2020-7-29, end: 2020-8-3
Heatwave 19 of 26: 
 start: 2020-8-19, end: 20

You will see an output showing the dates of 26 detected heatwaves in the 2017-2021 period. The following plot makes these dates easier to visualise, by shading the time periods in which heatwaves were detected on the plot produced in the previous section. You can use the toggle above the plot to change the visibility of the detection zones.

In [46]:
output_file("heatwaves_MY_2017_2021_clim_full.html")

# Select the desired data and prepare it for plotting
tag = "MY"
series = "2017-2021"

working_dict = heatwaves[tag][series]
clim_dict = heatwaves[tag]["full"]

data = ColumnDataSource(
    data=dict(
        t=working_dict["time_series"].time,
        SST=working_dict["time_series"],
        thresh=clim_dict["xmhw_thresh"].thresh[
            working_dict["time_series"].time.dt.dayofyear - 1
        ],
        seas=clim_dict["xmhw_thresh"].seas[
            working_dict["time_series"].time.dt.dayofyear - 1
        ],
    )
)

# Detect heatwaves in the data and record the dates in which they occur
mhws = detect(
    working_dict["time_series"],
    clim_dict["xmhw_thresh"].thresh,
    clim_dict["xmhw_thresh"].seas,
)

t_values = np.union1d(mhws.index_start.values, mhws.index_end.values)

# Prepare the plotting space and toolbar
tools = [BoxZoomTool(), WheelZoomTool(), PanTool(), ResetTool()]

p = figure(
    title=(
        f"MHW detection [dataset: {tag}, {working_dict['start_year']} - {working_dict['end_year']}; "
        f"climatology: {tag}, {clim_dict['start_year']} - {clim_dict['end_year']}]"
    ),
    height=400,
    width=1000,
    tools=tools,
    x_axis_type="datetime",
    x_axis_label="Date",
    y_axis_label=r"\[ SST \, [^\circ C] \]",
    y_range=(10, 32),
)

p.toolbar.autohide = True

# Plot SST, threshold and climatology data
p_sst = p.line("t", "SST", source=data, line_width=1.5, color="black")

p_thresh = p.line("t", "thresh", source=data, line_width=1.5, color=c_blue)
p_seas = p.line(
    "t", "seas", source=data, line_width=1.5, color="black", line_dash="dashed"
)

# Add hovertools to the plot
p.add_tools(
    HoverTool(
        renderers=[p_thresh],
        tooltips=[
            ("Date", "@t{%F}"),
            ("Threshold", "@thresh"),
            ("SST", "@SST"),
            ("Climatology", "@seas"),
        ],
        mode="vline",
        formatters={"@t": "datetime"},
        toggleable=False,
    )
)

# For each start and end date of a heatwave, fill the zone bounded above by SST
# and to the left and right by the start and end dates, respectively
MHW_zones = []
for t1, t2 in zip(*[iter(t_values)] * 2):
    # Calculate the dates, length and index of the heatwave
    start_date = working_dict["time_series"].time[int(t1)].dt.date.values
    end_date = working_dict["time_series"].time[int(t2 + 1)].dt.date.values
    length = int(t2) - int(t1) + 1
    index = int(t_values.tolist().index(t1) / 2 + 1)

    # Prepare the data to plot
    data = ColumnDataSource(
        data=dict(
            x=working_dict["time_series"].time[int(t1) : int(t2 + 1)],
            y1=working_dict["time_series"][int(t1) : int(t2 + 1)],
            y2=np.full(length, 10),
        )
    )

    # Fill the required area
    v_area = p.varea(
        "x", "y1", "y2", source=data, color=c_blue, alpha=0.6, hover_alpha=0.9
    )

    # Record the zone for later use
    MHW_zones.append(v_area)

    # Add a hovertool to the zone
    p.add_tools(
        HoverTool(
            renderers=[v_area],
            tooltips=[
                ("Index", f"{index}"),
                ("Dates", f"{start_date} - {end_date}"),
                ("Duration", f"{length} days"),
            ],
            formatters={"@start": "datetime", "@end": "datetime"},
            toggleable=False,
        ),
    )

# Create a toggle switch and for detection zones and add JavaScript
MHW_switch = Switch(active=True)

MHW_switch.js_on_change(
    "active",
    CustomJS(
        args=dict(MHW_zones=MHW_zones, MHW_switch=MHW_switch),
        code="""for (var i = 0; i < MHW_zones.length; i++) { MHW_zones[i].visible = MHW_switch.active}""",
    ),
)

MHW_switch_div = Div(text="Toggle MHW detection zones:")

# Add the plot legend
legend = Legend(
    items=[
        (f"SST [MY]", [p_sst]),
        (f"Seasonal climatology [MY]", [p_seas]),
        (f"90% MHW detection threshold", [p_thresh]),
        (f"Heatwave detection zone", [MHW_zones[0]]),
    ],
    location="top_right",
    ncols=4,
)
p.add_layout(legend, "center")

# Compile the plot layout and display!
container = row(p, height=400, sizing_mode="stretch_width")
container.stylesheets.append(":host { width: 100%; }")

layout = column(row(MHW_switch_div, MHW_switch), container, sizing_mode="stretch_both")

show(layout)

**Detail on thresholds**

In Lesson 1 we discussed how increasing the detection threshold for heatwaves could both increase or decrease the number of heatwaves detected in a given time period. We will now see that in action by learning how to adjust the detection threshold in our code.

The `xmhw` package function threshold can be passed a keyword argument `pctile` setting the desired threshold. When this argument is not passed, as we have done up to this point, the default value `pctile = 90` is used. We will compare the effect of a very small change in the detection threshold.

The following plot shows the heatwaves detected between January 2017 and December 2021 using the MY dataset and full climatology using percentiles between 85% and 95%. Use the slider above the plot to adjust the percentile displayed.

_Note: This cell can take a while to run, due to the amount of data being processed._

In [47]:
output_file("threshold_detail_percentiles.html")

# Select data for the plot
tag = "MY"
series = "2017-2021"

working_dict = heatwaves[tag][series]
clim_dict = heatwaves[tag]["full"]

# Set the range of detection thresholds to visualise

threshold_percentile_min = 85
threshold_percentile_max = 95
threshold_percentile_step = 1

threshold_percentiles = range(
    threshold_percentile_min, threshold_percentile_max + 1, threshold_percentile_step
)

# Setup the basic plot, and add the SST and climatology graphs

tools = [BoxZoomTool(), WheelZoomTool(), PanTool(), ResetTool()]

p = figure(
    title=f"MHW Detection with varying percentile thresholds "
    f"[Climatology data: {tag}, {clim_dict['start_year']} - {clim_dict['end_year']}]",
    height=400,
    width=1000,
    tools=tools,
    x_axis_type="datetime",
    x_axis_label="Date",
    y_axis_label=r"\[ SST \, [^\circ C] \]",
    y_range=(10, 32),
)

sst_data = ColumnDataSource(
    data=dict(
        t=working_dict["time_series"].time,
        SST=working_dict["time_series"],
        seas=clim_dict["xmhw_thresh"].seas[
            working_dict["time_series"].time.dt.dayofyear - 1
        ],
    )
)

p_sst = p.line("t", "SST", source=sst_data, line_width=1.5, color="black")
p_seas = p.line(
    "t", "seas", source=sst_data, line_width=1.5, color="black", line_dash="dashed"
)

# Run through the percentiles in the defined range. For each, plot the threshold
# graph, as well as the time periods of detected MHWs for that threshold

p_thresh = {}
for pctile in threshold_percentiles:
    # Initially, only the plots corresponding to the lowest percentile are visible

    visible = True if pctile == threshold_percentile_min else False

    # Compute the threshold for the given percentile

    clim = threshold(clim_dict["time_series"], pctile=pctile)

    # Collect the threshold data and plot the threshold graph

    mhw_data = ColumnDataSource(
        data=dict(
            t=sst_data.data["t"],
            SST=sst_data.data["SST"],
            thresh=clim.thresh[working_dict["time_series"].time.dt.dayofyear - 1],
            seas=sst_data.data["seas"],
        )
    )

    p_thresh[f"{pctile}"] = {}
    p_thresh[f"{pctile}"]["thresh_plot"] = p.line(
        "t",
        "thresh",
        source=mhw_data,
        line_width=1.5,
        color=c_blue,
        visible=visible,
    )

    # Add a hovertool to the threshold plot capturing SST, climatology and threshold data

    p.add_tools(
        HoverTool(
            renderers=[p_thresh[f"{pctile}"]["thresh_plot"]],
            tooltips=[
                ("Date", "@t{%F}"),
                ("Threshold", "@thresh"),
                ("SST", "@SST"),
                ("Climatology", "@seas"),
            ],
            mode="vline",
            formatters={"@t": "datetime"},
            toggleable=False,
        )
    )

    # Detect the MHWs for the given percentile

    mhws = detect(
        working_dict["time_series"],
        clim.thresh,
        clim.seas,
    )

    # Collect the start and end times for the detected MHWs

    t_values = np.union1d(mhws.index_start.values, mhws.index_end.values)

    # For each pair of start and end times, fill the area bounded above by the SST graph
    # and on the left and right by the start and end times

    MHW_zones = []
    for t1, t2 in zip(*[iter(t_values)] * 2):
        start_date = working_dict["time_series"].time[int(t1)].dt.date.values
        end_date = working_dict["time_series"].time[int(t2 + 1)].dt.date.values
        length = int(t2) - int(t1) + 1
        index = int(t_values.tolist().index(t1) / 2 + 1)

        data = ColumnDataSource(
            data=dict(
                x=working_dict["time_series"].time[int(t1) : int(t2 + 1)],
                y1=working_dict["time_series"][int(t1) : int(t2 + 1)],
                y2=np.full(length, 10),
            )
        )

        # Plots the wanted filled area

        v_area = p.varea(
            "x",
            "y1",
            "y2",
            source=data,
            color=c_blue,
            alpha=0.6,
            hover_alpha=0.9,
            visible=visible,
        )

        # Collect the MHW zone plot for later reference

        MHW_zones.append(v_area)

        # Add a hovertool to the MHW zones capturing index, dates and duration

        p.add_tools(
            HoverTool(
                renderers=[v_area],
                tooltips=[
                    ("Index", f"{index}"),
                    ("Dates", f"{start_date} - {end_date}"),
                    ("Duration", f"{length} days"),
                ],
                toggleable=False,
            ),
        )

    # Collect all MHW zone plots for later reference

    p_thresh[f"{pctile}"]["MHW_zones"] = MHW_zones

# Define a slider to select which threshold graph and MHW detection zones to view

thresh_select = Slider(
    start=threshold_percentile_min,
    end=threshold_percentile_max,
    step=threshold_percentile_step,
    value=(threshold_percentile_min + threshold_percentile_max) // 2,
    title="Threshold percentile",
)

# Link a change in the slider value to the visibility of relevant plots (JavaScript)

thresh_select.js_on_change(
    "value",
    CustomJS(
        args=dict(slider=thresh_select, plots=p_thresh),
        code="""
            for (let plot in plots) {
                plots[plot]["thresh_plot"].visible = false; 
                for (let zone in plots[plot]["MHW_zones"]) {
                    plots[plot]["MHW_zones"][zone].visible = false
                    }
                }; 
            plots[slider.value.toString()]["thresh_plot"].visible = true; 
            for (let zone in plots[slider.value.toString()]["MHW_zones"]) {
                plots[slider.value.toString()]["MHW_zones"][zone].visible = true
                }
            """,
    ),
)

# Define and format the plot legend

legend = Legend(
    items=[
        ("SST", [p_sst]),
        ("Climatology", [p_seas]),
        ("Threshold", [p_thresh[f"{threshold_percentile_min}"]["thresh_plot"]]),
        (
            "Heatwave detection zone",
            [p_thresh[f"{threshold_percentile_min}"]["MHW_zones"][0]],
        ),
    ],
    location="top_right",
    orientation="horizontal",
)
p.add_layout(legend, "center")

# Define and format the full plot container and layout

container = row(p, height=400, sizing_mode="stretch_width")
container.stylesheets.append(":host { width: 100%; }")

layout = column(thresh_select, container, sizing_mode="stretch_both")

# Display the plot!

show(layout)

Compare the outputs for different thresholds.

* How many heatwaves were detected in total in each case?
* As the detection threshold increases, find examples of:
    * A heatwave that turns into two distinct heatwaves;
    * A heatwave that is no longer detected.


You can rerun the code with different values of `threshold_percentile_min`, `threshold_percentile_max` and `threshold_percentile_step` to compare even more detection thresholds. Be aware that the code computes and plots every threshold percentile individually, so it can take a long time to run.

**Detail on MHW detection**

As well as the detection threshold, the definition of marine heatwaves also has two other parameters: minimum duration of temperature exceedance above the threshold, with a default period of 5 days; and a discreteness parameter, with a default of 2 days, which is used to determine if two nearby temperature exceedance events should be counted as one heatwave or multiple.

The `xmhw` package allows for the alteration of the latter two parameters by passing optional keyword arguments to the detect function. Passing the argument `minDuration` changes the duration parameter, and passing the argument `maxGap` changes the discreteness parameter.

The following plot analyses changing the parameter `minDuration` away from its default value of 5. For each duration between 3 and 7, it computes the heatwaves detected in the 2017 - 2021 period using the MY data over a full reference period, and plots the results.

In [48]:
output_file("detection_detail_durations.html")

# Select data for the plot
tag = "MY"
series = "2017-2021"

working_dict = heatwaves[tag][series]
clim_dict = heatwaves[tag]["full"]

# Set the range of MHW durations to visualise

MHW_duration_min = 3
MHW_duration_max = 7
MHW_duration_step = 1

MHW_durations = range(MHW_duration_min, MHW_duration_max + 1, MHW_duration_step)

# Setup the basic plot, and add the SST, climatology and threshold graphs

tools = [BoxZoomTool(), WheelZoomTool(), PanTool(), ResetTool()]

p = figure(
    title=f"MHW Detection with varying minimum durations "
    f"[Climatology data: {tag}, {clim_dict['start_year']} - {clim_dict['end_year']}]",
    height=400,
    width=1000,
    tools=tools,
    x_axis_type="datetime",
    x_axis_label="Date",
    y_axis_label=r"\[ SST \, [^\circ C] \]",
    y_range=(10, 32),
)

data = ColumnDataSource(
    data=dict(
        t=working_dict["time_series"].time,
        SST=working_dict["time_series"],
        seas=clim_dict["xmhw_thresh"].seas[
            working_dict["time_series"].time.dt.dayofyear - 1
        ],
        thresh=clim_dict["xmhw_thresh"].thresh[
            working_dict["time_series"].time.dt.dayofyear - 1
        ],
    )
)

p_sst = p.line("t", "SST", source=data, line_width=1.5, color="black")
p_seas = p.line(
    "t",
    "seas",
    source=data,
    line_width=1.5,
    color="black",
    line_dash="dashed",
)
p_thresh = p.line(
    "t",
    "thresh",
    source=data,
    line_width=1.5,
    color=c_blue,
)

# Add a hovertool to the threshold plot capturing SST, climatology and threshold data

p.add_tools(
    HoverTool(
        renderers=[p_thresh],
        tooltips=[
            ("Date", "@t{%F}"),
            ("Threshold", "@thresh"),
            ("SST", "@SST"),
            ("Climatology", "@seas"),
        ],
        mode="vline",
        formatters={"@t": "datetime"},
        toggleable=False,
    )
)

# Run through the percentiles in the defined range. For each, plot the threshold
# graph, as well as the time periods of detected MHWs for that threshold

p_durations = {}
for duration in MHW_durations:
    # Initially, only the plots corresponding to the lowest percentile are visible

    visible = True if duration == MHW_duration_min else False

    # Detect the MHWs for the given percentile

    mhws = detect(
        working_dict["time_series"],
        clim_dict["xmhw_thresh"].thresh,
        clim_dict["xmhw_thresh"].seas,
        minDuration=duration,
    )

    # Collect the start and end times for the detected MHWs

    t_values = np.union1d(mhws.index_start.values, mhws.index_end.values)

    # For each pair of start and end times, fill the area bounded above by the SST graph
    # and on the left and right by the start and end times

    MHW_zones = []
    for t1, t2 in zip(*[iter(t_values)] * 2):
        start_date = working_dict["time_series"].time[int(t1)].dt.date.values
        end_date = working_dict["time_series"].time[int(t2 + 1)].dt.date.values
        length = int(t2) - int(t1) + 1
        index = int(t_values.tolist().index(t1) / 2 + 1)

        data = ColumnDataSource(
            data=dict(
                x=working_dict["time_series"].time[int(t1) : int(t2 + 1)],
                y1=working_dict["time_series"][int(t1) : int(t2 + 1)],
                y2=np.full(length, 10),
            )
        )

        # Plots the wanted filled area

        v_area = p.varea(
            "x",
            "y1",
            "y2",
            source=data,
            color=c_blue,
            alpha=0.6,
            hover_alpha=0.9,
            visible=visible,
        )

        # Collect the MHW zone plot for later reference

        MHW_zones.append(v_area)

        # Add a hovertool to the MHW zones capturing index, dates and duration

        p.add_tools(
            HoverTool(
                renderers=[v_area],
                tooltips=[
                    ("Index", f"{index}"),
                    ("Dates", f"{start_date} - {end_date}"),
                    ("Duration", f"{length} days"),
                ],
                toggleable=False,
            ),
        )

    # Collect all MHW zone plots for later reference

    p_durations[f"{duration}"] = MHW_zones

# Define a slider to select which threshold graph and MHW detection zones to view

thresh_select = Slider(
    start=MHW_duration_min,
    end=MHW_duration_max,
    step=MHW_duration_step,
    value=(MHW_duration_min + MHW_duration_max) // 2,
    title="Minimum MHW duration",
)

# Link a change in the slider value to the visibility of relevant plots (JavaScript)

thresh_select.js_on_change(
    "value",
    CustomJS(
        args=dict(slider=thresh_select, plots=p_durations),
        code="""
            for (let plot in plots) {
                for (let zone in plots[plot]) {
                    plots[plot][zone].visible = false
                    }
                }; 
            for (let zone in plots[slider.value.toString()]) {
                plots[slider.value.toString()][zone].visible = true
                }
            """,
    ),
)

# Define and format the plot legend

legend = Legend(
    items=[
        ("SST", [p_sst]),
        ("Climatology", [p_seas]),
        ("Threshold", [p_thresh]),
        ("Heatwave detection zones", [p_durations[f"{MHW_duration_min}"][0]]),
    ],
    location="top_right",
    orientation="horizontal",
)
p.add_layout(legend, "center")

# Define and format the full plot container and layout

container = row(p, height=400, sizing_mode="stretch_width")
container.stylesheets.append(":host { width: 100%; }")

layout = column(thresh_select, container, sizing_mode="stretch_both")

# Display the plot!

show(layout)

Analyse the plots as the minimum duration parameter rises.
* How many heatwaves are detected in each case?
* Find an example of a heatwave that is detected with a shorter threshold, but not with a longer one.


Analysing the impact of changing the discreteness parameter can be done in the same way, except instead passing different values of the parameter `maxGap` to the `detect` function.

#### Lesson 6: The impact of climatology on marine heatwave detection

The previous lesson showed that the definition of marine heatwaves is very sensitive to the choice of data, even if the way you set climatology and threshold values for that dataset is the same. Therefore great care should be taken not only in ensuring you have accurate and reliable data, but also that you understand if the data you select is suitable for the experiment you want to conduct (spatial resolution, data frequency, and so on).

In this lesson, we will fix a dataset and look more deeply into the relationship between the reference period for climatology and the detection period for marine heatwaves. Specifically, you will:


* Appreciate that climatology from from the same data source over different reference periods affects the detection of marine heatwaves;
* Understand that climatic trends influence year-on-year comparisons of marine heatwaves;
* Appreciate that the length of time period used to define climatology can impact of heatwaves detected.

**Analysing the density of marine heatwaves detected**

From now on we will focus on using the MY dataset. As you may have started to realise in the previous lessons, the detection of heatwaves seems sensitive to the reference period used to define climatology. The goal of the next two lessons is to make this observation more concrete, and present ways to mitigate some of this sensitivity.

We will look at heatwave detection over a number of years, adjusting the climatology reference period. While the plots produced in the previous lessons are very useful for extracting data about specific heatwaves, they become messy and cluttered when visualising longer periods of time. Instead, we will extract the two main features they convey into a bar chart: number of heatwaves per year, and total duration of heatwaves per year.


The following cell calculates all the heatwaves detected by the MY data, using climatology based on its full range (1982 - 2021), and calculates the number per year, as well as the total duration per year.

In [49]:
from collections import Counter

# Select the dataset
tag = "MY"
series = "full"

working_dict = heatwaves[tag][series]

# Detect heatwaves
mhws = detect(
    working_dict["time_series"],
    working_dict["xmhw_thresh"].thresh,
    working_dict["xmhw_thresh"].seas,
)

# Count the number of heatwave occurences by year
starts_by_year = Counter(mhws.time_start.dt.year.values)

# Count the number of heatwave days per year
heatwave_days = mhws.groupby("time_start.year").sum(dim="events")

Next, we plot this information as a bar graph showing both number of heatwaves per year (left axis), and number of heatwave days per year (right axis).

In [50]:
tag = "MY"
series = "full"

output_file(f"MHW_density_{tag}_{series}.html")

working_dict = heatwaves[tag][series]

# Prepare the data from the previous cell for plotting
data = ColumnDataSource(
    data=dict(
        t=list(starts_by_year.keys()),
        nums=list(starts_by_year.values()),
        days=heatwave_days.duration.values,
    )
)

# Prepare the plotting space and toolbar
p = figure(
    title=f"Heatwave detections per year "
    f"[dataset: {tag}, {working_dict['start_year']} - {working_dict['end_year']}]",
    height=400,
    width=1000,
    x_axis_label="Year",
    y_axis_label="Number of heatwaves detected",
    tools=[],
    y_range=(0, 1.05 * max(data.data["nums"])),
)

p.extra_y_ranges["days"] = Range1d(0, 1.05 * max(data.data["days"]))

p.toolbar.autohide = True

# Plot the bar chart
p_nums = p.vbar(
    x=dodge("t", -0.2, range=p.x_range),
    top="nums",
    source=data,
    width=0.375,
    color=c_blue,
    alpha=0.6,
    hover_alpha=1,
)
p_days = p.vbar(
    x=dodge("t", 0.2, range=p.x_range),
    top="days",
    source=data,
    width=0.375,
    color=c_red,
    alpha=0.6,
    hover_alpha=1,
    y_range_name="days",
)

p.add_tools(
    HoverTool(
        renderers=[p_nums],
        tooltips=[("Year", "@t"), ("# Heatwaves", "@nums")],
        toggleable=False,
    ),
    HoverTool(
        renderers=[p_days],
        tooltips=[("Year", "@t"), ("# Heatwave days", "@days")],
        toggleable=False,
    ),
)

ax2 = LinearAxis(y_range_name="days", axis_label="Number of heatwave days detected")
p.add_layout(ax2, "right")

legend = Legend(
    items=[("Number of heatwaves", [p_nums]), ("Number of heatwave days", [p_days])],
    location="top_left",
)
p.add_layout(legend, "center")

# Set the plot layout and show!
container = row(p, height=400, sizing_mode="stretch_both")
container.stylesheets.append(":host { width: 100%; }")

show(container)

Based on this chart, it is tempting to conclude that marine heatwaves density and duration has increased in recent years, but this does not capture the full story. In previous lessons we calculated that MY data has an upwards trend. Since the detection threshold is the same whether we're looking at heatwaves in 1982 or 2021, we'd expect more heatwave detections when sea temperatures are, on average, higher. Therefore it's not possible to conclude from this plot whether the density of heatwaves has increased or not.

It is a common issue in climatic studies to recognise and account for climatic trends. Often different solutions are required based on the research question. In the remainder of this lesson and the next, we will present different ways of thinking about this issue in the context of marine heatwaves, and discuss additional problems some solutions can present.

**The stability of data versus the stability of heatwave detection**

The first idea we present is to base marine heatwave detection on solid long-term data. This can be useful for more qualitative overviews over longer periods, or used to assess or balance-out variability in high-resolution or high-frequency data (see section 2.2 of Hobday et al.).

As discussed in previous lessons, there is no universally-accepted climate data record (CDR) for sea surface temperature (SST) data. Nevertheless, using the multi-year reprocessed (MY) dataset for SST gives us reliable, robust and stable data over long periods. The following plot compares marine heatwave detection using two different 30-year reference periods of MY data based on typical CDR ranges: 1982 - 2011 and 1992 - 2021.

In [61]:
tag = "MY"

output_file(f"MHW_density_{tag}_CDRs.html")

# Setup the figure and interaction buttons
p = figure(
    title=f"Heatwave detections per year [dataset: {tag}]",
    height=400,
    width=1000,
    x_axis_label="Year",
    y_axis_label="Number of heatwaves detected",
    y_range=(0, 7.5),
    tools=[],
)

p.extra_y_ranges["days"] = Range1d(0, 160)

p.toolbar.autohide = True

labels = ["1982 - 2011", "1992 - 2021"]
rbg = RadioButtonGroup(labels=labels, active=0)

# Run through the two 30-year periods, 1982-2011 and 1992-2021, detect heatwaves in
# the full MY dataset for each climatology, and plot the result.
nums = []
days = []
for name, start_year, end_year in zip(["CDR_1", "CDR_2"], [1982, 1992], [2011, 2021]):
    create_TS_slice(tag, name, start_year, end_year)

    working_dict = heatwaves[tag][name]

    clim = threshold(working_dict["time_series"])
    mhws = detect(heatwaves[tag]["full"]["time_series"], clim.thresh, clim.seas)

    starts_by_year = Counter(mhws.time_start.dt.year.values)

    heatwave_days = mhws.groupby("time_start.year").sum(dim="events")

    data = ColumnDataSource(
        data=dict(
            t=list(starts_by_year.keys()),
            nums=list(starts_by_year.values()),
            days=heatwave_days.duration.values,
        )
    )

    p_nums = p.vbar(
        x=dodge("t", -0.2, range=p.x_range),
        top="nums",
        source=data,
        width=0.375,
        color=c_blue,
        alpha=0.6,
        hover_alpha=1,
        visible=True if name == "CDR_1" else False,
    )

    p_days = p.vbar(
        x=dodge("t", 0.2, range=p.x_range),
        top="days",
        source=data,
        width=0.375,
        color=c_red,
        alpha=0.6,
        hover_alpha=1,
        visible=True if name == "CDR_1" else False,
        y_range_name="days",
    )

    p.add_tools(
        HoverTool(
            renderers=[p_nums],
            tooltips=[("Year", "@t"), ("# Heatwaves", "@nums")],
            toggleable=False,
        ),
        HoverTool(
            renderers=[p_days],
            tooltips=[("Year", "@t"), ("# Heatwave days", "@days")],
            toggleable=False,
        ),
    )

    days.append(p_days)
    nums.append(p_nums)

# Add right-hand axis and legend
ax2 = LinearAxis(y_range_name="days", axis_label="Number of heatwave days detected")
p.add_layout(ax2, "right")

legend = Legend(
    items=[("Number of heatwaves", nums), ("Number of heatwave days", days)],
    location="top_left",
)
p.add_layout(legend, "center")

# Add JavaScript interactivity
rbg.js_on_change(
    "active",
    CustomJS(
        args=dict(rbg=rbg, days=days, nums=nums),
        code="""for (let i = 0; i < days.length; i++) """
        """{days[i].visible = rbg.active == i; nums[i].visible = rbg.active == i}""",
    ),
)

# Compile the plot layout and show!
container = row(p, height=400, sizing_mode="stretch_both")
container.stylesheets.append(":host { width: 100%; }")

rbg_div = Div(text="Select climatology period:")

layout = column(row(rbg_div,rbg), p, sizing_mode="stretch_width")

show(layout)

The two plots demonstrate that care must be taken even when choosing long term data. When focusing on particular years, it is clear that even when switching between two closely-related reference periods, there can be quite drastic changes in heatwave detection. In this sense, the definition of marine heatwaves can be seen as unstable.

Between the above two reference periods, only one year showed an increase in detections when using the period 1992 - 2021. Which year? Does this suggest anything about that year? What other data could you look at to find out more?

**Using disjoint climatic reference and detection periods**

The next method we present bases the detection of marine heatwaves in a given time period on climatology generated from a disjoint reference period. In this example, we will detect heatwaves in the 25-year period from 1997-2021 using climatology from the 15-year reference period 1982-1996. The idea of doing this is to reduce bias by separating out the dataset used for observation to the dataset used for measurement, a common technique in machine learning, for example.

A clear downside is the use of shorter reference periods, which can be more unstable due to the influence of data of individual years. This is something that should always be acknowledged when performing a trade-off such as this.

The following cell computes climatology from the reference period 1982-1996, then detects heatwaves using this climatology in the period 1997-2021.

In [52]:
from collections import Counter

tag = "MY"

# Generate the climatology
clim_series_name = "1982-1996"
create_TS_slice(tag, clim_series_name, 1982, 1996)
clim = threshold(heatwaves[tag][clim_series_name]["time_series"])

# Generate the time series slice for detection
detect_series_name = "1997+"
create_TS_slice(tag, detect_series_name, 1997, 2021)
detect_dict = heatwaves[tag][detect_series_name]

# Detect heatwaves
mhws = detect(detect_dict["time_series"], clim.thresh, clim.seas)

# Count the number of heatwave occurences by year
starts_by_year = Counter(mhws.time_start.dt.year.values)

# Count the number of heatwave days per year
heatwave_days = mhws.groupby("time_start.year").sum(dim="events")

The following graph displays the results.

In [53]:
output_file(f"MHW_density_1997_plus_using_1982_1996.html")

# Prepare the data from the previous cell for plotting
data = ColumnDataSource(
    data=dict(
        t=list(starts_by_year.keys()),
        nums=list(starts_by_year.values()),
        days=heatwave_days.duration.values,
    )
)

# Prepare the plotting space and toolbar
p = figure(
    title=f"Heatwave detections per year [dataset: {tag}, 1982 - 1996]",
    height=400,
    width=1000,
    x_axis_label="Year",
    y_axis_label="Number of heatwaves detected",
    tools=[],
    y_range=(0, 1.05 * max(data.data["nums"])),
)

p.extra_y_ranges["days"] = Range1d(0, 1.05 * max(data.data["days"]))

p.toolbar.autohide = True

# Plot the bar chart
p_nums = p.vbar(
    x=dodge("t", -0.2, range=p.x_range),
    top="nums",
    source=data,
    width=0.375,
    color=c_blue,
    alpha=0.6,
    hover_alpha=1,
)
p_days = p.vbar(
    x=dodge("t", 0.2, range=p.x_range),
    top="days",
    source=data,
    width=0.375,
    color=c_red,
    alpha=0.6,
    hover_alpha=1,
    y_range_name="days",
)

p.add_tools(
    HoverTool(
        renderers=[p_nums],
        tooltips=[("Year", "@t"), ("# Heatwaves", "@nums")],
        toggleable=False,
    ),
    HoverTool(
        renderers=[p_days],
        tooltips=[("Year", "@t"), ("# Heatwave days", "@days")],
        toggleable=False,
    ),
)

ax2 = LinearAxis(y_range_name="days", axis_label="Number of heatwave days detected")
p.add_layout(ax2, "right")

legend = Legend(
    items=[("Number of heatwaves", [p_nums]), ("Number of heatwave days", [p_days])],
    location="top_left",
)
p.add_layout(legend, "center")

# Set the plot layout and show!
container = row(p, height=400, sizing_mode="stretch_both")
container.stylesheets.append(":host { width: 100%; }")

show(container)

In the next lesson we will present and analyse a variation of this method, basing the detection of heatwaves in a given year on data available up to that year.

#### Lesson 7: The stability of rolling climatology

In the previous lesson, we started to see how climatic trends can impact the detection of marine heatwaves. In this lesson, we will explore just one of the many potential ideas to provide a year-to-year analysis of marine heatwaves while reducing the impact of long-term climatic trends. The concept that we will explore is to use a climatology reference period which depends on the year in which you want to study heatwaves.

Importantly, we will analyse the potential caveats and biases that can creep in to analyses such as these. An awareness of these, and more generally an awareness of the method of thinking through biases, is a useful skill when trying to address difficult questions such as this.

**Using variable climatologies to increase stability of heatwave detection**

For this method, we take idea of using disjoint reference and detection periods from the previous lesson, and aim to make year-on-year comparisons of heatwaves more stable. We will investigate the incidence of heatwaves for a given year using all the available information up to that year, but not beyond. For example, to detect heatwaves in 2007, we will use a climatology generated from the years 1982 up to 2006.

To ensure we're using enough data, we begin the experiment in the year 1997, giving us 15 years of data from 1982. The following cell generates all the slices of the MY time series data that we will use, and calculates the appropriate climatology and thresholds. This cell processes a lot of data, so can take a little while to run.

In [54]:
tag = "MY"

for year in range(1997, 2022):
    series_name = f"up to {year}"
    create_TS_slice(tag, series_name, 1982, year-1)
    create_TS_slice(tag, f"{year}", year)
    working_dict = heatwaves[tag][series_name]
    working_dict["xmhw_thresh"] = threshold(working_dict["time_series"])
    working_dict["xmhw_detect"] = detect(
        heatwaves[tag][f"{year}"]["time_series"],
        working_dict["xmhw_thresh"].thresh,
        working_dict["xmhw_thresh"].seas,
    )

The following cell then plots the results.

In [55]:
tag = "MY"

output_file(f"MHW_density_{tag}_data_upto_year.html")

# Organise the data for plotting

t = [year for year in range(1997, 2022)]
nums = [len(heatwaves[tag][f"up to {year}"]["xmhw_detect"].events) for year in t]
days = [
    heatwaves[tag][f"up to {year}"]["xmhw_detect"].duration.sum(dim="events").values
    for year in t
]

data = ColumnDataSource(data=dict(t=t, nums=nums, days=days))

# Prepare the plotting space and toolbar
p = figure(
    title=f"Heatwave detections per year [dataset: {tag}, variable reference period]",
    height=400,
    width=1000,
    x_axis_label="Year",
    y_axis_label="Number of heatwaves detected",
    tools=[],
    y_range=(0, 1.1 * max(data.data["nums"])),
)

p.extra_y_ranges["days"] = Range1d(0, 1.05 * max(data.data["days"]))

p.toolbar.autohide = True

# Plot the bar chart
p_nums = p.vbar(
    x=dodge("t", -0.2, range=p.x_range),
    top="nums",
    source=data,
    width=0.375,
    color=c_blue,
    alpha=0.6,
    hover_alpha=1,
)
p_days = p.vbar(
    x=dodge("t", 0.2, range=p.x_range),
    top="days",
    source=data,
    width=0.375,
    color=c_red,
    alpha=0.6,
    hover_alpha=1,
    y_range_name="days",
)

p.add_tools(
    HoverTool(
        renderers=[p_nums],
        tooltips=[("Year", "@t"), ("# Heatwaves", "@nums")],
        toggleable=False,
    ),
    HoverTool(
        renderers=[p_days],
        tooltips=[("Year", "@t"), ("# Heatwave days", "@days")],
        toggleable=False,
    ),
)

ax2 = LinearAxis(y_range_name="days", axis_label="Number of heatwave days detected")
p.add_layout(ax2, "right")

legend = Legend(
    items=[("Number of heatwaves", [p_nums]), ("Number of heatwave days", [p_days])],
    location="top_left",
)
p.add_layout(legend, "center")

# Set the plot layout and show!
container = row(p, height=400, sizing_mode="stretch_both")
container.stylesheets.append(":host { width: 100%; }")

show(container)

Before continuing, think about some of the issues of this method, including the biases that may have appeared.

**Successive 15 year trend gradients**

In the previous exercise we detected heatwaves in a given year using data available up to that year, as a way to make year-to-year comparisons of heatwaves. One important consideration of this method is that there is inherent bias in detecting heatwaves in two consecutive years using climatology from two different reference periods. In this case, we are able to examine the difference between the two climatologies since they are closely related to each other. This can help mitigate the bias in making comparisons by quantifying how stable the definition of climatology is for different reference periods.

Before we examine how to perform this analysis, another issue with the previous exercise is that detection in 2021 has 39 years of data available, while detection in 1997 only has 15 years of data available. This has the potential to be another source of bias. A solution is to base heatwave detection for each year on the same length range of data, for example 15 years. The following cell and bar chart are produced in exactly the same way as the previous exercise, except changing the definition of the time series slices so that, fore example, detection in 1997 is based off a 15-year climatology from 1982-1996, while detection in 2021 is also based off a 15-year climatology, except from 2006-2020.

In [56]:
tag = "MY"

for year in range(1997, 2022):
    series_name = f"{year-15} up to {year}"
    create_TS_slice(tag, series_name, year-15, year-1)
    create_TS_slice(tag, f"{year}", year)
    working_dict = heatwaves[tag][series_name]
    working_dict["xmhw_thresh"] = threshold(working_dict["time_series"])
    working_dict["xmhw_detect"] = detect(
        heatwaves[tag][f"{year}"]["time_series"],
        working_dict["xmhw_thresh"].thresh,
        working_dict["xmhw_thresh"].seas,
    )

The following chart displays the results. Compare it to the previous chart.

In [57]:
tag = "MY"

output_file(f"MHW_density_{tag}_15_year_rolling.html")

# Organise the data for plotting

t = [year for year in range(1997, 2022)]
nums = [
    len(heatwaves[tag][f"{year-15} up to {year}"]["xmhw_detect"].events) for year in t
]
days = [
    heatwaves[tag][f"{year - 15} up to {year}"]["xmhw_detect"]
    .duration.sum(dim="events")
    .values
    for year in t
]

data = ColumnDataSource(data=dict(t=t, nums=nums, days=days))

# Prepare the plotting space and toolbar
p = figure(
    title=f"Heatwave detections per year "
    f"[dataset: {tag}, rolling 15 year reference period]",
    height=400,
    width=1000,
    x_axis_label="Year",
    y_axis_label="Number of heatwaves detected",
    tools=[],
    y_range=(0, 1.1 * max(data.data["nums"])),
)

p.extra_y_ranges["days"] = Range1d(0, 1.05 * max(data.data["days"]))

p.toolbar.autohide = True

# Plot the bar chart
p_nums = p.vbar(
    x=dodge("t", -0.2, range=p.x_range),
    top="nums",
    source=data,
    width=0.375,
    color=c_blue,
    alpha=0.6,
    hover_alpha=1,
)
p_days = p.vbar(
    x=dodge("t", 0.2, range=p.x_range),
    top="days",
    source=data,
    width=0.375,
    color=c_red,
    alpha=0.6,
    hover_alpha=1,
    y_range_name="days",
)

p.add_tools(
    HoverTool(
        renderers=[p_nums],
        tooltips=[("Year", "@t"), ("# Heatwaves", "@nums")],
        toggleable=False,
    ),
    HoverTool(
        renderers=[p_days],
        tooltips=[("Year", "@t"), ("# Heatwave days", "@days")],
        toggleable=False,
    ),
)

ax2 = LinearAxis(y_range_name="days", axis_label="Number of heatwave days detected")
p.add_layout(ax2, "right")

legend = Legend(
    items=[("Number of heatwaves", [p_nums]), ("Number of heatwave days", [p_days])],
    location="top_left",
)
p.add_layout(legend, "center")

# Set the plot layout and show!
container = row(p, height=400, sizing_mode="stretch_both")
container.stylesheets.append(":host { width: 100%; }")

show(container)

**Stability of rolling 15 year climatologies**

We return to quantifying the stability of successive 15-year climatologies. One method of doing this is to calculate and compare the regression trends of each 15-year reference period of SST data. This can also be compared to longer-term SST trends, for example the full 40-year range of MY data, to assess the impact that using shorter reference periods has on the study.

The following cell computes and stores the trend gradient of each 15-year reference period of SST data used in the previous part of this lesson.

In [58]:
# Access the desired data
tag = "MY"

slopes = []

for year in range(1997,2022):
    series = f"{year-15} up to {year}"
    working_dict = heatwaves[tag][series]

    # Compute the mean temperature per year by first grouping the time series per year,
    # then taking the mean along the time dimension.
    annual_mean = working_dict["time_series"].groupby('time.year').mean('time')

    # Use the stats Python package to compute a regression model and store the regression slope
    regress = stats.linregress(annual_mean.year, annual_mean)
    slopes.append(regress.slope)

The following graph plots the 15-year trend slopes stored in `slopes`. It also plots the 40-year trend in the MY SST data as a horizonal dashed line for reference.

In [59]:
output_file(f"15_year_rolling_climatology_trends.html")

# Prepare the data from the previous cell for plotting
data = ColumnDataSource(
    data=dict(
        t=[year for year in range(1997,2022)],
        slopes = slopes
    )
)

# Prepare the plotting space and toolbar
p = figure(
    title=f"Stability of 15 year time series trends [dataset: {tag}]",
    height=400,
    width=1000,
    x_axis_label="Year",
    y_axis_label="Trend gradient of SST time series",
)

p.toolbar.autohide = True

# Plot the bar chart

p_circ = p.circle('t','slopes', source=data, size=7, color=c_blue)
p_line = p.line('t','slopes',source=data, line_width=1.5, color=c_blue)
p_line2 = p.hspan(y=0.03338862216942306, line_width=1.5, line_dash="dashed", color="black")

p.add_tools(
    HoverTool(renderers=[p_circ, p_line],
              tooltips=[("Year", "@t"), ("Slope", "@slopes{0.000}")],
              toggleable = False,
              mode = "vline"
              )
)

legend = Legend(
    items=[("Trend of rolling 15 year SST ending ending at given year", [p_circ, p_line]),
           ("Trend of 40 year MY SST", [p_line2])],
    location="top_right",
)
p.add_layout(legend, "center")

# Set the plot layout and show!
container = row(p, height=400, sizing_mode="stretch_both")
container.stylesheets.append(":host { width: 100%; }")

show(container)

Whether the variation in the successive 15-year climatology reference periods seen in the graph is significant or not depends on the scope and specifics of the study you are looking to conduct.